In [1]:
import torch

dataset_path = "/home/rafael/Área de trabalho/Linux/graph_dataset.pt"
dataset = torch.load(dataset_path)

print(f"Graphs loaded: {len(dataset)}")

/tmp/ipykernel_6542/3346811239.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dataset_path)


Graphs loaded: 1920


In [2]:
import sys
sys.path.append("/media/rafael/HD/orguel_ml_library")
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from orguel_ml import BalanceClassWeights, GraphGPSNetwork

# Setup
epochs = 60
batch_size = 1
learning_rate = 0.0025
weight_decay = 1e-4
smoothing_factor = 0.2
label_smoothing = 0.1
test_size = 0.1

# split the dataset:
train_data, validation_data = train_test_split(dataset, test_size=test_size, shuffle=True, random_state=42)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class_weights = BalanceClassWeights(train_data, device, smoothing_factor)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

model = GraphGPSNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=5, mode="triangular")
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, min_lr=1e-5, threshold=1e-5, verbose=True)

/home/rafael/anaconda3/envs/dl_torch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [3]:
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# TensorBoard writer
writer = SummaryWriter(log_dir="TensorBoard")

# Training loop with Cross Entropy clearly shown
for epoch in range(epochs):
    model.train()
    acumulateLoss = 0
    correctPredictions = 0
    totalNodesProcessed = 0
    
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch)
        
        # cross_entropy expects raw logits:
        loss = F.cross_entropy(output, batch.y, weight=class_weights, label_smoothing=label_smoothing)
        
        # backpropagation
        loss.backward()
        optimizer.step()
        #scheduler.step() # CyclicLR
        
        # compute accuracy
        acumulateLoss += loss.item()
        predictedClass = output.argmax(dim=1)
        correctPredictions += (predictedClass == batch.y).sum().item()
        totalNodesProcessed += batch.num_nodes
    
    # Computes epoch-wide accuracy & loss
    trainAccuracy = correctPredictions / totalNodesProcessed
    averageTrainLoss = acumulateLoss / len(train_loader)
    
    # Evaluate clearly:
    model.eval()
    acumulateLoss = 0
    correctPredictions = 0
    totalNodesProcessed = 0

    with torch.no_grad():
        for batch in validation_loader:
            batch = batch.to(device)
            output = model(batch)
            loss = F.cross_entropy(output, batch.y, weight=class_weights)
            acumulateLoss += loss.item()
            predictedClass = output.argmax(dim=1)
            correctPredictions += (predictedClass == batch.y).sum().item()
            totalNodesProcessed += batch.num_nodes

    ValidationAccuracy = correctPredictions / totalNodesProcessed
    averageValidationLoss = acumulateLoss / len(validation_loader)
    
    # Adjust learning rate based on validation loss
    scheduler.step(averageValidationLoss) # ReduceLROnPlateau
    
    # Log learning rate
    currentLearningRate = optimizer.param_groups[0]['lr']
    writer.add_scalar("LearningRate", currentLearningRate, epoch)
    
    # Logging
    writer.add_scalar("Loss/train", averageTrainLoss, epoch)
    writer.add_scalar("Loss/val", averageValidationLoss, epoch)
    writer.add_scalar("Accuracy/train", trainAccuracy, epoch)
    writer.add_scalar("Accuracy/val", ValidationAccuracy, epoch)
    print(f"Epoch {epoch+1} | Train Loss: {averageTrainLoss:.4f} | Train Accuracy: {trainAccuracy:.2f} | Validation Loss: {averageValidationLoss:.4f} | Validation Accuracy: {ValidationAccuracy:.2f}| Learning Rate: {currentLearningRate:.6f}")

writer.close()

print("\nTraining complete. You can now launch TensorBoard:")

Epoch 1 | Train Loss: 1.0071 | Train Accuracy: 0.72 | Validation Loss: 0.3747 | Validation Accuracy: 0.87| Learning Rate: 0.002500
Epoch 2 | Train Loss: 0.7448 | Train Accuracy: 0.91 | Validation Loss: 0.2724 | Validation Accuracy: 0.92| Learning Rate: 0.002500
Epoch 3 | Train Loss: 0.6885 | Train Accuracy: 0.94 | Validation Loss: 0.2217 | Validation Accuracy: 0.95| Learning Rate: 0.002500
Epoch 4 | Train Loss: 0.6557 | Train Accuracy: 0.96 | Validation Loss: 0.1858 | Validation Accuracy: 0.97| Learning Rate: 0.002500
Epoch 5 | Train Loss: 0.6541 | Train Accuracy: 0.96 | Validation Loss: 0.1702 | Validation Accuracy: 0.97| Learning Rate: 0.002500
Epoch 6 | Train Loss: 0.6425 | Train Accuracy: 0.96 | Validation Loss: 0.1946 | Validation Accuracy: 0.96| Learning Rate: 0.002500
Epoch 7 | Train Loss: 0.6513 | Train Accuracy: 0.96 | Validation Loss: 0.1528 | Validation Accuracy: 0.98| Learning Rate: 0.002500
Epoch 8 | Train Loss: 0.6249 | Train Accuracy: 0.97 | Validation Loss: 0.1916 | Val

In [ ]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir TensorBoard

In [4]:
# Save model to a file
save_path = "/home/rafael/Área de trabalho/Linux/GraphGPSNetwork.pt"
torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")

Model saved to /home/rafael/Área de trabalho/Linux/GraphGPSNetwork.pt


In [7]:
from collections import Counter

# number of labels of each class
labels = [data.y.tolist() for data in dataset]
flat_labels = [item for sublist in labels for item in sublist]
print(Counter(flat_labels))

Counter({0: 152448, 1: 133504, 3: 130224, 2: 7648})
